In [11]:
data_file = '/Users/irv033/Downloads/data/thetao_Omon_IPSL-CM5A-LR_historicalMisc_r1i1p3_185001-189912.nc'

#data_file = '/Users/irv033/Downloads/data/thetao-maps_Omon_IPSL-CM5A-LR_historicalMisc_r1i1p3_all.nc'
#depth_file = '/Users/irv033/Downloads/data/deptho_fx_IPSL-CM5A-LR_historicalMisc_r0i0p0.nc'

In [12]:
import iris

## Case 1: No depth file

This case also applies for curvilinear grids, since calculating the vertical mean field requires regridding to a rectilinear grid first, rendering the depth file useless.

In [14]:
data_cube = iris.load_cube(data_file, 'sea_water_potential_temperature')

In [15]:
print data_cube

sea_water_potential_temperature          (time: 600; depth: 31; cell index along second dimension: 149; cell index along first dimension: 182)
     Dimension coordinates:
          time                                  x           -                                      -                                      -
          depth                                 -           x                                      -                                      -
          cell index along second dimension     -           -                                      x                                      -
          cell index along first dimension      -           -                                      -                                      x
     Auxiliary coordinates:
          latitude                            -           -                                      x                                      x
          longitude                           -           -                                      x     

In [9]:
depth_cube = iris.load_cube(depth_file)

/Applications/anaconda/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'deptho'
  warnings.warn(message % (variable_name, nc_var_name))


In [10]:
print depth_cube

sea_floor_depth_below_geoid / (m)   (cell index along second dimension: 149; cell index along first dimension: 182)
     Dimension coordinates:
          cell index along second dimension                           x                                      -
          cell index along first dimension                            -                                      x
     Auxiliary coordinates:
          latitude                                                    x                                      x
          longitude                                                   x                                      x
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_IPSL-CM5A-LR_historicalMisc_r0i0p0.nc...
          branch_time: 1870.0
          cmor_version: 2.7.1
          comment: Ocean bathymetry.   Reported here is the sea floor depth for present day....
          contact: ipsl-c